<!--NOTEBOOK_HEADER-->
*This notebook contains material from [cbe-virtual-laboratory](https://jckantor.github.io/cbe-virtual-laboratory);
content is available [on Github](https://github.com/jckantor/cbe-virtual-laboratory.git).*


<!--NAVIGATION-->
< [9.0 Interfacing with Laboratory Equipment](https://jckantor.github.io/cbe-virtual-laboratory/09.00-Interfacing_with_Laboratory_Equipment.html) | [Contents](toc.html) | [9.2 Stepper Motor Control via I2C](https://jckantor.github.io/cbe-virtual-laboratory/09.02-Stepper_Motor_Control_via_I2C.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe-virtual-laboratory/blob/master/docs/09.01-Communicating _via_TCP.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe-virtual-laboratory/09.01-Communicating _via_TCP.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>

# 9.1 Communicating via TCP

Show how to set up Particle Argon as a TCP Server.

## 9.1.1 Particle CLI

### 9.1.1.1 Installation

In [1]:
%%capture
!bash <( curl -sL https://particle.io/install-cli )

### 9.1.1.2 Utility functions

In [2]:
import re
import subprocess

# regular expression to strip ansi control characters
ansi = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

# decode byte string and strip ansi control characters
def decode_bytes(byte_string):
    if isinstance(byte_string, bytes):
        result = byte_string.decode("utf-8")
    return ansi.sub("", result)

# streamline call to the particle-cli
def particle(args, particle_cli="/root/bin/particle"):
    process = subprocess.run([particle_cli] + args,
                             stdout=subprocess.PIPE,
                             stderr=subprocess.PIPE)
    process.stdout = decode_bytes(process.stdout)
    process.stderr = decode_bytes(process.stderr)
    return process

### 9.1.1.3 Login to Particle

In [3]:
import getpass

# prompt for username and password
username = getpass.getpass(prompt="Username: ")
password = getpass.getpass(prompt="Password: ")

# attempt login
output = particle(["login", "--username", username, "--password", password])

# report results
if output.returncode:
    print(f"Return code = {output.returncode}")
    print(output.stderr)
else:
    print(output.stdout)

Username: ··········
Password: ··········
> Successfully completed login!



### 9.1.1.4 Select a device

The following cell downloads a list of all user devices and creates a list of device names. Here we choose the first name in the list for the rest of this notebook. If this is not the device to be used, then modify this cell accordingly.

In [4]:
devices = [line.split()[0] for line in particle(["list"]).stdout.splitlines()]
device_name = devices[0]
print(particle(["list", device_name]).stdout)

jck_argon_01 [e00fce68eaceb1faa7cf7193] (Argon) is online



## 9.1.2 Project Considerations

Using the Particle Device OS, it is straightforward to setup a TCP server which a user can access via standard Windows, MacOS, or Linux utilities. 

## 9.1.3 Implementation

MacOS

    nc -4 -n 192.168.xxx.xxx pppp

### 9.1.3.1 Create Project

In [5]:
print(particle(["project", "create", "--name", "myproject", "."]).stdout)

Initializing project in directory myproject...
> A new project has been initialized in directory myproject



### 9.1.3.2 Change working directory

The Particle CLI assumes one is working in the top  project directory.

In [6]:
%cd /content/myproject

/content/myproject


### 9.1.3.3 Create source file

In [81]:
%%writefile src/myproject.ino

/* 
Demonstrate bidirectional communication with Particle using TCPServer and
client. After flashing this firmware, check the Particle console for the local
IP address. Connect to the local IP address using netcat

    nc -4 -n 192.168.xxx.xxx 23

Type H and L will turn the on-board LED on and off.
*/

TCPServer server = TCPServer(23);
TCPClient client;

const int led = D7;
char c;

void setup() {
    server.begin();
    Particle.publish("Server IP", WiFi.localIP().toString().c_str(), PRIVATE);
    pinMode(led, OUTPUT);
}

void loop() {
    if (client.connected()) {
        while (client.available()) {
            /* get next char */
            c = client.read();
            if ('H' == c) {
                server.print("Turn LED on.");
                digitalWrite(led, HIGH);
            } else if ('L' == c) {
                server.print("Turn LED off.");
                digitalWrite(led, LOW);
            } else if ('S' == c) {
                if (digitalRead(led)) {
                    server.print("LED is on.");
                } else {
                    server.print("LED is off.");
                }
            } else if ('X' == c) {
                server.print("Close client connection.");
                client.stop();
            } else {
                server.write(c);
            }
        }
    } else {
        /* get next client */
        client = server.available();
        if (client.connected()) {
            server.println("Enter: ");
            server.println("   H to turn LED on");
            server.println("   L to turn LED off");
            server.println("   S for LED status");
            server.println("   X to close client connection");
        }
    }
}


Overwriting src/myproject.ino


### 9.1.3.4 Compiling

In [82]:
print(particle(["compile", "argon", "--saveTo", "myproject.bin"]).stdout)


Compiling code for argon

Including:
    src/myproject.ino
    project.properties

attempting to compile firmware
downloading binary from: /v1/binaries/5f95ca4704393a21fcca5c6c
saving to: myproject.bin
Memory use:
   text	   data	    bss	    dec	    hex	filename
  11692	    108	   1144	  12944	   3290	/workspace/target/workspace.elf

Compile succeeded.
Saved firmware to: /content/myproject/myproject.bin



### 9.1.3.5 Flash firmware

In [83]:
print(particle(["flash", device_name, "myproject.bin"]).stdout)

Including:
    myproject.bin

attempting to flash firmware to your device jck_argon_01
Flash device OK: Update started

Flash success!



<!--NAVIGATION-->
< [9.0 Interfacing with Laboratory Equipment](https://jckantor.github.io/cbe-virtual-laboratory/09.00-Interfacing_with_Laboratory_Equipment.html) | [Contents](toc.html) | [9.2 Stepper Motor Control via I2C](https://jckantor.github.io/cbe-virtual-laboratory/09.02-Stepper_Motor_Control_via_I2C.html) ><p><a href="https://colab.research.google.com/github/jckantor/cbe-virtual-laboratory/blob/master/docs/09.01-Communicating _via_TCP.ipynb"> <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a><p><a href="https://jckantor.github.io/cbe-virtual-laboratory/09.01-Communicating _via_TCP.ipynb"> <img align="left" src="https://img.shields.io/badge/Github-Download-blue.svg" alt="Download" title="Download Notebook"></a>